In [1]:
# todo
# generate an interactive plot for our pc

In [1]:
from field_util import Field, CartCoord
from vis_util import texture_opacity

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates.matrix_utilities import rotation_matrix
import numpy as np
from scipy import ndimage
import copy
from matplotlib.colors import LogNorm as LogNorm
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import pandas as pd
import os

from matplotlib.backends.backend_pdf import PdfPages

In [2]:
cosmo = FlatLambdaCDM(H0=100* u.km / u.s / u.Mpc, Om0=0.315) # Metin original
mycoord = CartCoord(150.14205192829834, 2.224237689411875, cosmo) # Used in Cosmic Birth

In [3]:
cosmo

FlatLambdaCDM(H0=100.0 km / (Mpc s), Om0=0.315, Tcmb0=0.0 K, Neff=3.04, m_nu=None, Ob0=None)

In [4]:
import numpy as np
from mayavi import mlab

#viz_type = 'png'  # 'png' or 'x3d' or 'ipy'
#mlab.init_notebook('png')
#mlab.init_notebook(viz_type)



In [5]:
R_smooth = 4 * u.Mpc

# clamato
clamato_path = '../Data/CLAMATO/map_2020_v0.bin'
delta_clm_real = np.fromfile(clamato_path, dtype='double').reshape(68, 56, 876).transpose(2, 0, 1)
delta_clm_real = delta_clm_real[::1,::-1,::1] 
# this reversal is due to the coordinate in clamato is 
#    x - ra     y - dec
#    0 - 149.89 0 - 2.07
#    68 - 150.4 56 - 2.50  
# consistent with https://github.com/kheegan/preheating2019
x_0, y_0, z_0= mycoord.orig_to_box(149.89, 2.07, 2.05)
CLAMATO_field = Field(x_0, y_0-34*u.Mpc, z_0, 0.5*u.Mpc, delta_clm_real)
res = CLAMATO_field.smooth(R_smooth)

In [6]:
_

FlatLambdaCDM(H0=100.0 km / (Mpc s), Om0=0.315, Tcmb0=0.0 K, Neff=3.04, m_nu=None, Ob0=None)

In [28]:
red_min = 2.178
red_max = 2.409
ra_min = 149.89
ra_max = 150.40

dec_min = 2.07
dec_max = 2.50

red_mean = 2.30
ra_mean = 150.15
dec_mean = 2.30

x_mean = 3855*u.Mpc
y_mean = 0*u.Mpc
z_mean = 0*u.Mpc

_, y_min, z_min = mycoord.orig_to_box(ra_max, dec_min, 2.300) # Note y axis is reversed
_, y_max, z_max = mycoord.orig_to_box(ra_min, dec_max, 2.300) # Note y axis is reversed



/Users/dongcz/Work/COSMOS_Preheating/Code/field_util.py:37: UserWarning: Warning: Index out of boundary
  warnings.warn("Warning: Index out of boundary")


In [16]:
skewer = pd.read_csv("../Catalogs/CLAMATO_skewer.csv")
sightline = pd.read_csv("../Catalogs/list_tomo_input_2020_mod.txt", sep = '\t')

In [17]:
with open("../Catalogs/all_specz_DM_v1.dat") as f:
    lines = f.readlines()
    headers = lines[0].split()
    data = []
    for line in lines[1:]:
        entry = line.split()
        data.append(entry)
gal_df = pd.DataFrame(data, columns = headers)
gal_df['RA'] = gal_df['RA'].astype(float)
gal_df['Dec'] = gal_df['Dec'].astype(float)
gal_df['zspec'] = gal_df['zspec'].astype(float)

x, y, z = mycoord.orig_to_box(gal_df['RA'], gal_df['Dec'], gal_df['zspec'])

gal_df['x'] = x.value
gal_df['y'] = y.value
gal_df['z'] = z.value

gal_df_ = gal_df
gal_df_ = gal_df_[(gal_df_['x'] > x_min) & (gal_df_['x'] < x_max)]
gal_df_ = gal_df_[(gal_df_['y'] > y_min) & (gal_df_['y'] < y_max)]
gal_df_ = gal_df_[(gal_df_['z'] > z_min) & (gal_df_['z'] < z_max)]

In [82]:
gal_mem = pd.read_csv("../Catalogs/COSTCO_I_iter.csv")

In [83]:
# for interactive
x_min, x_max = 3810*u.Mpc, 3900*u.Mpc

clamato_plot = CLAMATO_field.clip_with_coord([x_min, y_min, z_min], [x_max, y_max, z_max])

/Users/dongcz/Work/COSMOS_Preheating/Code/field_util.py:37: UserWarning: Warning: Index out of boundary
  warnings.warn("Warning: Index out of boundary")


In [84]:
from mayavi import mlab

mlab.clf()

viz_type = 'x3d'  # 'png' or 'x3d' or 'ipy'
#mlab.init_notebook('png')
mlab.init_notebook(viz_type, local=False)

#cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.15, vmin = -0.2, contours = [-0.1], 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont00')
#cbar = mlab.colorbar(label_fmt="%-#.2f")
#cbar.lut_mode = 'jet'
#cbar.reverse_lut = True
#
#cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.15, vmin = -0.2, contours = [-0.0], 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont01')
#cbar = mlab.colorbar(label_fmt="%-#.2f")
#cbar.lut_mode = 'jet'
#cbar.reverse_lut = True
#
#cont3d = mlab.contour3d(np.zeros_like(clamato_plot.field_data), opacity = 0.0, colormap = "jet", vmax = 0.15, vmin = -0.2, 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont_bkg')


cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.25, vmin = -0.15, contours = [-0.1, 0.0, 0.1], 
                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont')
otl = mlab.outline(cont3d, extent = [3810. , 3900. ,  -17.35665149,   16.95483991, -10.38353398, 18.56887978],  color=(0.2,0.2,0.2), line_width=1, name = 'box')



cbar = mlab.colorbar(label_fmt="%-#.2f")
cbar.lut_mode = 'jet'
cbar.reverse_lut = True

#cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.15, vmin = -0.2, contours = [0.1], 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont02')


#for gal in sightline.iloc:
#    skewer_ = skewer[skewer['tomo_id'] == gal['TomoID']]
#    skewer_ = skewer_[(skewer_['x_new'] < x_max.value) & (skewer_['x_new'] > x_min.value)]
#    skewer_ = skewer_[(skewer_['y_new'] < y_max.value) & (skewer_['y_new'] > y_min.value)]
#    skewer_ = skewer_[(skewer_['z_new'] < z_max.value) & (skewer_['z_new'] > z_min.value)]
#    if len(skewer_) > 0:
#        pass
#        mlab.points3d(skewer_['x_new'], skewer_['y_new'], skewer_['z_new'], color = (1,1,1), scale_factor = 0.9, mode = 'cylinder', opacity = 0.5)

#dummy_cont3d = mlab.contour3d(np.zeros_like(clamato_plot.field_data), opacity = 0,
#                        extent = [red_min, red_max, ra_max, ra_min, dec_min, dec_max])

mlab.points3d(gal_mem['x'], gal_mem['y'], gal_mem['z'], scale_factor = 1.5, color = (1, 0, 0), name = 'gal_member')
mlab.points3d(gal_df_['x'], gal_df_['y'], gal_df_['z'], color = (1, 1, 1), scale_factor = 1, name = 'gal_other')

#ax=mlab.axes(cont3d, color=(1,1,1),xlabel='x (Mpc/h)', x_axis_visibility = True, y_axis_visibility = True, z_axis_visibility = True, nb_labels = 11)
#ax.axes.label_format='%.1f'

labelscale = 1


for xtmp in np.arange(x_max.value, x_min.value, -5)[1:]:
    sky_coord = mycoord.original_pos(xtmp, y_mean.value, z_mean.value)
    xstring = '{0:.3f}'.format(sky_coord.distance.value)    #mlab.text3d(zcom0+(zcom1-zcom0)/2., xtmp,-2, xstring, scale=labelscale, orient_to_camera=False,
    #            orientation = np.asarray([0,90.,180.]) )
    mlab.text3d(xtmp, y_max.value, z_min.value, xstring, scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]))
    mlab.text3d(x_mean.value, y_max.value+5, z_min.value, 'Redshift', scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]))
    
for ytmp in np.arange(y_max.value, y_min.value, -5)[1:]:
    sky_coord = mycoord.original_pos(x_mean.value, ytmp, z_mean.value)

    ystring = '{0:.2f}'.format(sky_coord.ra.value)
    #mlab.text3d(zcom0+(zcom1+zcom0)/2., -2,ytmp, ystring, scale=labelscale, orient_to_camera=False,
    #            orientation = np.asarray([0,90.,90.]) )
    mlab.text3d(x_max.value, ytmp, z_max.value, ystring, scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,45.]) )
    mlab.text3d(x_max.value, y_mean.value-0.5, z_max.value + 5, 'RA (deg)', scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]) )

for ztmp in np.arange(z_max.value, z_min.value, -5)[1:]:
    sky_coord = mycoord.original_pos(x_mean.value, y_mean.value, ztmp)

    zstring = '{0:.2f}'.format(sky_coord.dec.value)
    #mlab.text3d(zcom0+(zcom1+zcom0)/2., -2,ytmp, ystring, scale=labelscale, orient_to_camera=False,
    #            orientation = np.asarray([0,90.,90.]) )
    mlab.text3d(x_max.value, y_max.value, ztmp, zstring, scale=labelscale, orient_to_camera=False, orientation = np.asarray([180.,90.,90.]) )

    mlab.text3d(x_max.value, y_max.value + 4, z_mean.value + 2, 'Dec (deg)', scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]) )
#mlab.view(azimuth=60, elevation=60, distance=150, focalpoint=[gal_mem['x'][0], gal_mem['y'][0], gal_mem['z'][0]])
mlab.view(azimuth=30, elevation=75, distance=150, focalpoint=[gal_mem['x'][0], gal_mem['y'][0], gal_mem['z'][0]])
#mlab.view(azimuth=60, elevation=60, distance=150, focalpoint=[gal_mem['x'][0], gal_mem['y'][0], gal_mem['z'][0]])

mlab.savefig('../interactive/cont3d_cont.x3d')
mlab.show()

Notebook initialized with x3d backend.


In [85]:
# foe still figure
x_min, x_max = 3830*u.Mpc, 3880*u.Mpc

clamato_plot = CLAMATO_field.clip_with_coord([x_min, y_min, z_min], [x_max, y_max, z_max])

In [88]:

sky_coord = mycoord.original_pos(x_max.value, y_mean.value, z_mean.value)
xstring = '{0:.3f}'.format(sky_coord.distance.value)    #mlab.text3d(zcom0+(zcom1-zcom0)/2., xtmp,-2, xstring, scale=labelscale, orient_to_camera=False,
print(xstring)

2.326


In [79]:
from mayavi import mlab

mlab.clf()

viz_type = 'png'  # 'png' or 'x3d' or 'ipy'
#mlab.init_notebook('png')
mlab.init_notebook(viz_type, local=False)

#cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.15, vmin = -0.2, contours = [-0.1], 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont00')
#cbar = mlab.colorbar(label_fmt="%-#.2f")
#cbar.lut_mode = 'jet'
#cbar.reverse_lut = True
#
#cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.15, vmin = -0.2, contours = [-0.0], 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont01')
#cbar = mlab.colorbar(label_fmt="%-#.2f")
#cbar.lut_mode = 'jet'
#cbar.reverse_lut = True
#
#cont3d = mlab.contour3d(np.zeros_like(clamato_plot.field_data), opacity = 0.0, colormap = "jet", vmax = 0.15, vmin = -0.2, 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont_bkg')


cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.25, vmin = -0.15, contours = [-0.1, 0.0, 0.1], 
                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont')
otl = mlab.outline(cont3d, extent = [x_min.value, x_max.value ,  -17.35665149,   16.95483991, -10.38353398, 18.56887978],  color=(0.2,0.2,0.2), line_width=1, name = 'box')



cbar = mlab.colorbar(cont3d, label_fmt="%-#.2f")
cbar.lut_mode = 'jet'
cbar.reverse_lut = True

#cont3d = mlab.contour3d(clamato_plot.field_data, opacity = 0.3, colormap = "jet", vmax = 0.15, vmin = -0.2, contours = [0.1], 
#                        extent = [x_min.value, x_max.value, y_min.value, y_max.value, z_min.value, z_max.value], name = 'cont02')


#for gal in sightline.iloc:
#    skewer_ = skewer[skewer['tomo_id'] == gal['TomoID']]
#    skewer_ = skewer_[(skewer_['x_new'] < x_max.value) & (skewer_['x_new'] > x_min.value)]
#    skewer_ = skewer_[(skewer_['y_new'] < y_max.value) & (skewer_['y_new'] > y_min.value)]
#    skewer_ = skewer_[(skewer_['z_new'] < z_max.value) & (skewer_['z_new'] > z_min.value)]
#    if len(skewer_) > 0:
#        pass
#        mlab.points3d(skewer_['x_new'], skewer_['y_new'], skewer_['z_new'], color = (1,1,1), scale_factor = 0.9, mode = 'cylinder', opacity = 0.5)

#dummy_cont3d = mlab.contour3d(np.zeros_like(clamato_plot.field_data), opacity = 0,
#                        extent = [red_min, red_max, ra_max, ra_min, dec_min, dec_max])

mlab.points3d(gal_mem['x'], gal_mem['y'], gal_mem['z'], scale_factor = 1.5, color = (1, 0, 0), name = 'gal_member')
mlab.points3d(gal_df_['x'], gal_df_['y'], gal_df_['z'], color = (1, 1, 1), scale_factor = 1, name = 'gal_other')

#ax=mlab.axes(cont3d, color=(1,1,1),xlabel='x (Mpc/h)', x_axis_visibility = True, y_axis_visibility = True, z_axis_visibility = True, nb_labels = 11)
#ax.axes.label_format='%.1f'

labelscale = 1


#for xtmp in np.arange(x_max.value, x_min.value, -5)[1:]:
#    sky_coord = mycoord.original_pos(xtmp, y_mean.value, z_mean.value)
#    xstring = '{0:.3f}'.format(sky_coord.distance.value)    #mlab.text3d(zcom0+(zcom1-zcom0)/2., xtmp,-2, xstring, scale=labelscale, orient_to_camera=False,
#    #            orientation = np.asarray([0,90.,180.]) )
#    mlab.text3d(xtmp, y_max.value, z_min.value, xstring, scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]))
#    mlab.text3d(x_mean.value, y_max.value+5, z_min.value, 'Redshift', scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]))
    
for ytmp in np.arange(y_max.value, y_min.value, -5)[1:]:
    sky_coord = mycoord.original_pos(x_mean.value, ytmp, z_mean.value)

    ystring = '{0:.2f}'.format(sky_coord.ra.value)
    #mlab.text3d(zcom0+(zcom1+zcom0)/2., -2,ytmp, ystring, scale=labelscale, orient_to_camera=False,
    #            orientation = np.asarray([0,90.,90.]) )
    mlab.text3d(x_max.value, ytmp, z_max.value, ystring, scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,45.]) )
    mlab.text3d(x_max.value, y_mean.value-0.5, z_max.value + 5, 'RA (deg)', scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]) )

for ztmp in np.arange(z_max.value, z_min.value, -5)[1:]:
    sky_coord = mycoord.original_pos(x_mean.value, y_mean.value, ztmp)

    zstring = '{0:.2f}'.format(sky_coord.dec.value)
    #mlab.text3d(zcom0+(zcom1+zcom0)/2., -2,ytmp, ystring, scale=labelscale, orient_to_camera=False,
    #            orientation = np.asarray([0,90.,90.]) )
    mlab.text3d(x_max.value, y_max.value, ztmp, zstring, scale=labelscale, orient_to_camera=False, orientation = np.asarray([180.,90.,90.]) )

    mlab.text3d(x_max.value, y_max.value + 4, z_mean.value + 2, 'Dec (deg)', scale=labelscale, orient_to_camera=False,  orientation = np.asarray([180.,90.,90.]) )
#mlab.view(azimuth=60, elevation=60, distance=150, focalpoint=[gal_mem['x'][0], gal_mem['y'][0], gal_mem['z'][0]])
mlab.view(azimuth=0, elevation=90, distance=90, focalpoint=[gal_mem['x'][0], 5, 5])
#mlab.view(azimuth=60, elevation=60, distance=150, focalpoint=[gal_mem['x'][0], gal_mem['y'][0], gal_mem['z'][0]])

mlab.savefig('../interactive/cont3d_short_cont.x3d')


Notebook initialized with png backend.


In [56]:
cont3d?

Type:            IsoSurface
String form:     <mayavi.modules.iso_surface.IsoSurface object at 0x2c3836e50>
File:            ~/miniconda3/envs/mayavi/lib/python3.9/site-packages/mayavi/modules/iso_surface.py
Docstring:       <no docstring>
Class docstring:
Base class for the Mayavi modules.
    


In [20]:
    <Viewpoint id="obs" fieldOfView="5.23598790168762207e-01" position="3.98317318359943829e+03 7.80317864972799526e+01 3.70345103750007425e+01" description="Default View" orientation="3.31584432764682246e-01 5.74321084547335992e-01 7.48469809539767494e-01 -3.95737995889419869e+00" centerOfRotation="3.85769573803876710e+03 5.58734952559984777e+00 -1.78834639037737020e+00"/>
    <Viewpoint id="obso" fieldOfView="5.23598790168762207e-01" position="3.98317318359943829e+03 7.80317864972799526e+01 3.70345103750007425e+01" description="Default View" orientation="3.31584432764682246e-01 5.74321084547335992e-01 7.48469809539767494e-01 -3.95737995889419869e+00" centerOfRotation="3.85769573803876710e+03 5.58734952559984777e+00 -1.78834639037737020e+00"/>
    <Viewpoint id="obsx" fieldOfView="5.23598790168762207e-01" position="3.85769573803876710e+03 5.58734952559984777e+00 1.48211653609622630e+02" description="Default View" orientation="0.00000000000000000e+00 0.00000000000000000e+00 1.00000000000000000e+00 -0.00000000000000000e+00" centerOfRotation="3.85769573803876710e+03 5.58734952559984777e+00 -1.78834639037737020e+00"/>
    <Viewpoint id="obsy" fieldOfView="5.23598790168762207e-01" position="3.85769573803876710e+03 1.55587349525599848e+02 -1.78834639037736109e+00" description="Default View" orientation="0.00000000000000000e+00 7.07106781186547462e-01 7.07106781186547573e-01 -3.14159265358979312e+00" centerOfRotation="3.85769573803876710e+03 5.58734952559984777e+00 -1.78834639037737020e+00"/>
    <Viewpoint id="obsz" fieldOfView="5.23598790168762207e-01" position="4.00769573803876710e+03 5.58734952559984777e+00 -1.78834639037736109e+00" description="Default View" orientation="5.77350269189625842e-01 5.77350269189625953e-01 5.77350269189625620e-01 -4.18879020478639053e+00" centerOfRotation="3.85769573803876710e+03 5.58734952559984777e+00 -1.78834639037737020e+00"/>

Signature:
mlab.colorbar(
    object=None,
    title=None,
    orientation=None,
    nb_labels=None,
    nb_colors=None,
    label_fmt=None,
)
Docstring:
Adds a colorbar for the color mapping of the given object.

If the object has scalar data, the scalar color mapping is
represented. Elsewhere the vector color mapping is represented, if
available.
If no object is specified, the first object with a color map in the scene
is used.

**Keyword arguments**:

    :object: Optional object to get the color map from

    :title: The title string

    :orientation: Can be 'horizontal' or 'vertical'

    :nb_labels: The number of labels to display on the colorbar.

    :label_fmt: The string formater for the labels. This needs to be
                a formater for float number, eg '%.1f'.

    :nb_colors: The maximum number of colors displayed on the
                colorbar.
File:      ~/miniconda3/envs/mayavi/lib/python3.9/site-packages/mayavi/tools/decorations.py
Type:      function


# what to present in the 3d viz?
# 1. delta_F field -> mlab.contour3d
# 2. pc members -> mlab.points3d
# 3. sightline skewers -> mlab.plot3d